In [1]:
import pandas as pd
import datetime
import json
from datetime import timedelta

In [2]:
from couchbase.exceptions import CouchbaseException
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

In [10]:
# Configuración de Couchbase
endpoint = "couchbases://cb.rwijg3tkoqodgahr.cloud.couchbase.com"
username = "BDD_POLITICA"
password = "Admin_1234"  # Reemplazar con la contraseña real
bucket_name = "BDD_POLITICA"
scope_name = "OpinionPublica"
collection_name = "Protestas2022-2024"

In [11]:
# Couchbase connection setup
auth = PasswordAuthenticator(username, password)
options = ClusterOptions(auth)
options.apply_profile("wan_development")

cluster = Cluster(endpoint, options)
cluster.wait_until_ready(timedelta(seconds=5))
cb = cluster.bucket(bucket_name)
cb_coll = cb.scope(scope_name).collection(collection_name)

In [12]:
# Ruta del archivo JSON generado anteriormente
json_path = "protestas2022-2024.json"

# Cargar los datos en un DataFrame de Pandas
with open(json_path, "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

# Mostrar las primeras filas para verificar la carga
df.head()


,proyecto_id,nombre,fecha_presentacion,fecha_votacion,resultado,votos,partidos,tema,promotor
0,1,Rich where.,2023-06-25,2024-06-21,Archivado,"{'a_favor': 59, 'en_contra': 56, 'abstenciones...","{'CREO': 'A favor', 'SUMA': 'A favor', 'ID': '...",EducaciÃ³n,UNES
1,2,Employee benefit.,2024-06-03,2024-06-26,Archivado,"{'a_favor': 93, 'en_contra': 79, 'abstenciones...","{'ID': 'AbstenciÃ³n', 'RC': 'AbstenciÃ³n', 'PK...",Salud,CREO
2,3,Space affect.,2023-07-02,2024-11-19,Aprobado,"{'a_favor': 75, 'en_contra': 58, 'abstenciones...","{'RC': 'AbstenciÃ³n', 'ID': 'AbstenciÃ³n'}",Salud,SUMA
3,4,Increase wear.,2024-02-21,2024-03-03,Aprobado,"{'a_favor': 80, 'en_contra': 40, 'abstenciones...","{'RC': 'En contra', 'CREO': 'En contra', 'SUMA...",Justicia,SUMA
4,5,Your could.,2024-06-26,2024-12-09,Aprobado,"{'a_favor': 83, 'en_contra': 47, 'abstenciones...","{'RC': 'AbstenciÃ³n', 'SUMA': 'En contra', 'UN...",EducaciÃ³n,RC


In [13]:
# Ruta para guardar el CSV
csv_path = "protestas2022-2024.csv"

# Guardar el DataFrame en CSV
df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"Archivo CSV guardado en: {csv_path}")


Archivo CSV guardado en: protestas2022-2024.csv


In [14]:
# Subir el archivo CSV a Couchbase 
try:
    with open('protestas2022-2024.csv', 'r', encoding='utf-8') as f:
        csv_content = f.read()
    
    cb_coll.insert('protestas2022_2024_csv', {'content': csv_content})
    print("CSV content inserted into Couchbase.")
except CouchbaseException as e:
    print(f"Error inserting CSV content into Couchbase: {e}")
except Exception as e:
    print(f"General error: {e}")

CSV content inserted into Couchbase.
